In [ ]:
import pandas as pd
import numpy as np

from datetime import datetime
%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

In [ ]:
mpl.rcParams['figure.figsize'] = (16, 9)
pd.set_option('display.max_rows', 500)
sns.set(style = "darkgrid")

In [ ]:
df_plot = pd.read_csv('C:/Users/patka/enterprise_data_science-covid-19/data/processed/COVID_small_flat_table.csv', sep=';')
df_plot.head()

In [ ]:
plt.figure();
ax = df_plot.iloc[5:,:].set_index('date').plot()
plt.ylim(100, 3000000000)
ax.set_yscale('log')

## Dynamic Visualizations using Ploty

In [ ]:
import plotly.graph_objects as go
import plotly
plotly.__version__

In [ ]:
fig = go.Figure()

In [ ]:
country_list = ['Italy','US','Spain','Germany','Korea, South']
#Adding each country's trace using loop

for each in country_list:
    fig.add_trace(go.Scatter(x=df_plot.date,
                             y = df_plot[each],
                             mode = 'markers+lines',
                             opacity = 0.9,
                            line_width = 2,
                            marker_size = 4,
                            name = each))

# Customizing Figure

fig.update_layout(
    width = 1600,
    height = 1200,
    xaxis_title = "Time",
    yaxis_title = "confirmed infected people (Joh hopkins data set, log-scale)",
)
fig.update_yaxes(type = "log", range = [1.1,8.5])


fig.update_layout(xaxis_rangeslider_visible = True)

# To render in Chrome
#fig.show(renderer='chrome')

## Using Dashboard for choice of country

In [ ]:
import dash
from dash import dcc
from dash import html

app = dash.Dash()
app.layout = html.Div([
    
    html.Label('Multi-Select Country'),
    dcc.Dropdown(
        id = 'country_drop_down',
        options = [
            {'label': 'Italy', 'value': 'Italy'},
            {'label': 'US', 'value': 'US'},
            {'label': 'Mexico', 'value': 'Mexico'},
            {'label': 'Germany', 'value': 'Germany'},
            {'label': 'Korea, South', 'value': 'Korea, South'}
        ],
        value = ['US', 'Germany'],
        multi = True
    ),
    
    dcc.Graph(figure=fig, id = 'main_window_slope')
])

In [ ]:
from dash.dependencies import Input, Output

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value')])

def update_figure(country_list):
    
    traces = []
    for each in country_list:
        traces.append(dict(x=df_plot.date,
                          y= df_plot[each],
                          mode = 'markers+lines',
                          opacity= 0.9,
                          line_width =2,
                           marker_size = 4,
                            name = each
                          )
                     )
    return {
        'data' : traces,
        'layout': dict(
            width = 1600,
            height = 1200,
            xaxis_title = "Time",
            yaxis_title = "confirmed infected people (Joh hopkins data set, log-scale)",
            xaxis = {'tickangle': -45,
                    'nticks': 20,
                    'tickfont': dict(size = 14, color = "#7f7f7f"),
                    },
            yaxis = {'type': "log",
                    'range': '[1.1,5.5]'
                    }
        )
    }

In [ ]:
app.run_server(debug=True, use_reloader = False)